# **Save and restore models**
Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

code to create the model, and
the trained weights, or parameters, for the model
Sharing this data helps others understand how the model works and try it themselves with new data.

## Get an example dataset
We'll use the Fashion-MNIST dataset again to train our model to demonstrate saving weights. To speed up these demonstration runs, only use the first 1000 examples:

In [1]:
from __future__ import absolute_import, division, print_function

import os
import pathlib

import keras
import tensorflow

keras.__version__

'2.8.0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tensorflow.keras.datasets.fashion_mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

4431872/4422102 [==============================] - 0s 0us/step


### Define a model
Let's build a simple model we'll use to demonstrate saving and loading weights.

In [3]:
# Returns a short sequential model
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
  
    model.compile(optimizer='Adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  
    return model


# Create a basic model instance
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training
The primary use case is to automatically save checkpoints during and at the end of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

keras.callbacks.ModelCheckpoint is a callback that performs this task. The callback takes a couple of arguments to configure checkpointing.

### Checkpoint callback usage
Train the model and pass it the ModelCheckpoint callback:

In [4]:
checkpoint_path = os.path.join("training_1", "cp.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)
os.mkdir(checkpoint_dir)

# Create checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                              save_weights_only=True,
                                              verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2650 - accuracy: 0.5391
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 15ms/step - loss: 1.2319 - accuracy: 0.5530 - val_loss: 0.7681 - val_accuracy: 0.7150
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.6936 - accuracy: 0.7640
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.6862 - accuracy: 0.7640 - val_loss: 0.6611 - val_accuracy: 0.7630
Epoch 3/10
31/32 [============================>.] - ETA: 0s - loss: 0.5550 - accuracy: 0.8196
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.5605 - accuracy: 0.8180 - val_loss: 0.8609 - val_accuracy: 0.6920
Epoch 4/10
29/32 [==========================>...] - ETA: 0s - loss: 0.5286 - accuracy: 0.8254
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - lo

This creates a single collection of Keras checkpoint files that are updated at the end of each epoch:

In [5]:
!ls {checkpoint_dir}


checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different instance of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 3ms/step - loss: 2.4055 - accuracy: 0.0340
Untrained model, accuracy:  3.40%


Then load the weights from the checkpoint, and re-evaluate:



In [7]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 7ms/step - loss: 0.5814 - accuracy: 0.7950
Restored model, accuracy: 79.50%


## Checkpoint callback options
The callback provides several options to give the resulting checkpoints unique names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs:

In [8]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = os.path.join("training_2", "cp-{epoch:04d}.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)
os.mkdir(checkpoint_dir)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


Now, have a look at the resulting checkpoints (sorting by modification date):



In [16]:
# Sort the checkpoints by modification time.
checkpoints = pathlib.Path(checkpoint_dir).glob("*.ckpt")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
#latest = str(checkpoints[-1])
checkpoints

[]

Note: the default Keras format only saves the 5 most recent checkpoints.

To test, reset the model and load the latest checkpoint:

In [10]:
model = create_model()
model.load_weights(latest+".ckpt")
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

NameError: ignored

## What are these files?
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

One or more shards that contain your model's weights.
An index file that indicates which weights are stored in which shard.
If you are only training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

## Manually save weights
Above you saw how to load the weights into a model.

Manually saving the weights is just as simple, use the Model.save_weights method.

In [11]:
# Save the weights
os.mkdir("checkpoints")
model.save_weights(os.path.join("checkpoints", "my_checkpoint"))

# Restore the weights
model = create_model()
model.load_weights(os.path.join("checkpoints", "my_checkpoint"))

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 [==============================] - 0s 3ms/step - loss: 2.5911 - accuracy: 0.1310
Restored model, accuracy: 13.10%


## Save the entire model
The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model in Keras is very useful.

Keras provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob.

In [12]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 9ms/step - loss: 1.1277 - accuracy: 0.6140
Epoch 2/5
32/32 [==============================] - 0s 9ms/step - loss: 0.6979 - accuracy: 0.7530
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.5661 - accuracy: 0.8090
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.5158 - accuracy: 0.8150
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4860 - accuracy: 0.8260


Now recreate the model from that file:



In [13]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:



In [14]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


32/32 [==============================] - 0s 3ms/step - loss: 0.5958 - accuracy: 0.7790
Restored model, accuracy: 77.90%


This technique saves everything:

* The weight values
* The model's configuration(architecture)
* The optimizer configuration